# Check Optimizer params and do we make the final classifier (Linear layers) bigger?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Images/test_images.zip"

Streaming output truncated to the last 5000 lines.
  inflating: test_images/15082.jpg   
  inflating: __MACOSX/test_images/._15082.jpg  
  inflating: test_images/05291.jpg   
  inflating: __MACOSX/test_images/._05291.jpg  
  inflating: test_images/25141.jpg   
  inflating: __MACOSX/test_images/._25141.jpg  
  inflating: test_images/03838.jpg   
  inflating: __MACOSX/test_images/._03838.jpg  
  inflating: test_images/23530.jpg   
  inflating: __MACOSX/test_images/._23530.jpg  
  inflating: test_images/12935.jpg   
  inflating: __MACOSX/test_images/._12935.jpg  
  inflating: test_images/15096.jpg   
  inflating: __MACOSX/test_images/._15096.jpg  
  inflating: test_images/05285.jpg   
  inflating: __MACOSX/test_images/._05285.jpg  
  inflating: test_images/32429.jpg   
  inflating: __MACOSX/test_images/._32429.jpg  
  inflating: test_images/25155.jpg   
  inflating: __MACOSX/test_images/._25155.jpg  
  inflating: test_images/14388.jpg   
  inflating: __MACOSX/test_images/._14388.jpg  
  i

In [3]:
!unzip "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Images/train_images.zip"

Streaming output truncated to the last 5000 lines.
  inflating: train_images/01949.jpg  
  inflating: __MACOSX/train_images/._01949.jpg  
  inflating: train_images/28303.jpg  
  inflating: __MACOSX/train_images/._28303.jpg  
  inflating: train_images/17695.jpg  
  inflating: __MACOSX/train_images/._17695.jpg  
  inflating: train_images/06940.jpg  
  inflating: __MACOSX/train_images/._06940.jpg  
  inflating: train_images/20039.jpg  
  inflating: __MACOSX/train_images/._20039.jpg  
  inflating: train_images/19860.jpg  
  inflating: __MACOSX/train_images/._19860.jpg  
  inflating: train_images/27756.jpg  
  inflating: __MACOSX/train_images/._27756.jpg  
  inflating: train_images/28465.jpg  
  inflating: __MACOSX/train_images/._28465.jpg  
  inflating: train_images/07486.jpg  
  inflating: __MACOSX/train_images/._07486.jpg  
  inflating: train_images/26448.jpg  
  inflating: __MACOSX/train_images/._26448.jpg  
  inflating: train_images/06798.jpg  
  inflating: __MACOSX/train_images/._0679

In [4]:
!unzip "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Images/validation_images.zip"

Streaming output truncated to the last 5000 lines.
  inflating: validation_images/31652.jpg  
  inflating: __MACOSX/validation_images/._31652.jpg  
  inflating: validation_images/27030.jpg  
  inflating: __MACOSX/validation_images/._27030.jpg  
  inflating: validation_images/01949.jpg  
  inflating: __MACOSX/validation_images/._01949.jpg  
  inflating: validation_images/28303.jpg  
  inflating: __MACOSX/validation_images/._28303.jpg  
  inflating: validation_images/17695.jpg  
  inflating: __MACOSX/validation_images/._17695.jpg  
  inflating: validation_images/06940.jpg  
  inflating: __MACOSX/validation_images/._06940.jpg  
  inflating: validation_images/20039.jpg  
  inflating: __MACOSX/validation_images/._20039.jpg  
  inflating: validation_images/19860.jpg  
  inflating: __MACOSX/validation_images/._19860.jpg  
  inflating: validation_images/27756.jpg  
  inflating: __MACOSX/validation_images/._27756.jpg  
  inflating: validation_images/28465.jpg  
  inflating: __MACOSX/validation_

In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import sys
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

## Ensure TensorFlow is not used
import os
os.environ["USE_TF"] = "0"

# For reproducability
random_state = 42

# Changed this

In [6]:
train_data = "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Text/train.csv"
test_data = "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Text/test_5k.csv"
validation_data = "/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/Text/validation_5k.csv"

In [7]:
import pandas as pd

TRAIN_DATA = pd.read_csv(train_data)
VALIDATION_DATA = pd.read_csv(validation_data, index_col=0)
TEST_DATA = pd.read_csv(test_data, index_col=0)

In [8]:
TRAIN_DATA

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
0,this spongebob squarepants branded battery,2019-07-30 20:00:50,i.redd.it,https://preview.redd.it/f39wxxk8yhd31.jpg?widt...,4.0,33,mildlyinteresting,0.95,1,0,0
1,award for careless talk,2011-09-03 17:26:23,i.imgur.com,https://external-preview.redd.it/KgPHCi1u3fY5j...,1.0,14,propagandaposters,1.00,0,1,5
2,four aligned airplanes,2017-11-20 06:05:45,i.redd.it,https://preview.redd.it/88v9axk19phx.jpg?width...,24.0,198,confusing_perspective,0.98,0,2,2
3,columbus discovers the new world,2019-08-28 15:40:17,i.redd.it,https://preview.redd.it/x4wzpd0am7j31.jpg?widt...,5.0,318,fakehistoryporn,0.98,0,2,2
4,feed me drummmmssssssss,2014-05-09 13:23:59,i.imgur.com,https://external-preview.redd.it/yNN57loQnVhLk...,0.0,3,pareidolia,0.62,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...
33319,toothbrush holder loves its job,2018-08-27 03:49:30,i.redd.it,https://preview.redd.it/c7n0e4ot5ki11.jpg?widt...,1.0,11,pareidolia,0.92,0,2,2
33320,guess how many zika cases showed up at the oly...,2016-08-26 18:51:11,npr.org,https://external-preview.redd.it/wwdKH-S3jgOG_...,2.0,46,savedyouaclick,0.91,0,2,5
33321,a very fluffy cat,2016-04-16 03:04:55,i.imgur.com,https://external-preview.redd.it/iWOHMEGAS7goI...,11.0,208,photoshopbattles,0.95,1,0,0
33322,found this scared little guy in the milwaukee ...,2017-11-08 19:52:08,i.imgur.com,https://external-preview.redd.it/4fr6-DmioX6Xh...,0.0,6,pareidolia,0.80,0,2,2


# Getting image path from indexes

In [9]:
TRAIN_DATA['image_num'] = TRAIN_DATA.index.astype(str).str.zfill(5)
VALIDATION_DATA['image_num'] = VALIDATION_DATA.index.astype(str).str.zfill(5)
TEST_DATA['image_num'] = TEST_DATA.index.astype(str).str.zfill(5)

In [10]:
TRAIN_DATA

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label,image_num
0,this spongebob squarepants branded battery,2019-07-30 20:00:50,i.redd.it,https://preview.redd.it/f39wxxk8yhd31.jpg?widt...,4.0,33,mildlyinteresting,0.95,1,0,0,00000
1,award for careless talk,2011-09-03 17:26:23,i.imgur.com,https://external-preview.redd.it/KgPHCi1u3fY5j...,1.0,14,propagandaposters,1.00,0,1,5,00001
2,four aligned airplanes,2017-11-20 06:05:45,i.redd.it,https://preview.redd.it/88v9axk19phx.jpg?width...,24.0,198,confusing_perspective,0.98,0,2,2,00002
3,columbus discovers the new world,2019-08-28 15:40:17,i.redd.it,https://preview.redd.it/x4wzpd0am7j31.jpg?widt...,5.0,318,fakehistoryporn,0.98,0,2,2,00003
4,feed me drummmmssssssss,2014-05-09 13:23:59,i.imgur.com,https://external-preview.redd.it/yNN57loQnVhLk...,0.0,3,pareidolia,0.62,0,2,2,00004
...,...,...,...,...,...,...,...,...,...,...,...,...
33319,toothbrush holder loves its job,2018-08-27 03:49:30,i.redd.it,https://preview.redd.it/c7n0e4ot5ki11.jpg?widt...,1.0,11,pareidolia,0.92,0,2,2,33319
33320,guess how many zika cases showed up at the oly...,2016-08-26 18:51:11,npr.org,https://external-preview.redd.it/wwdKH-S3jgOG_...,2.0,46,savedyouaclick,0.91,0,2,5,33320
33321,a very fluffy cat,2016-04-16 03:04:55,i.imgur.com,https://external-preview.redd.it/iWOHMEGAS7goI...,11.0,208,photoshopbattles,0.95,1,0,0,33321
33322,found this scared little guy in the milwaukee ...,2017-11-08 19:52:08,i.imgur.com,https://external-preview.redd.it/4fr6-DmioX6Xh...,0.0,6,pareidolia,0.80,0,2,2,33322


In [11]:
VALIDATION_DATA

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label,image_num
8637,not as heartwarming as it could have been anth...,2019-09-19 17:48:33,lifestyle.clickhole.com,https://external-preview.redd.it/850kBbKdgMKfz...,0.0,15,theonion,0.86,0,2,1,08637
20669,other discussions,2013-12-09 23:58:43,NaN,http://i.dailymail.co.uk/i/pix/2013/12/09/arti...,NaN,0,psbattle_artwork,NaN,0,2,4,20669
13179,on reflection oc,2015-01-12 17:14:55,i.imgur.com,https://external-preview.redd.it/tiFw8Ggb178E4...,0.0,3,pareidolia,0.67,0,2,2,13179
20565,viet congo setting booby trap,2019-02-27 09:57:07,i.redd.it,https://preview.redd.it/mj81gkh533j21.jpg?widt...,4.0,14,fakehistoryporn,0.80,0,2,2,20565
15504,chief has some happy shoulder armour,2013-02-20 01:00:02,i.imgur.com,https://external-preview.redd.it/kqqIlnAOZxzxA...,0.0,6,pareidolia,1.00,0,2,2,15504
...,...,...,...,...,...,...,...,...,...,...,...,...
29923,albino squirrel spotted outside the national g...,2019-03-25 14:50:50,i.redd.it,https://preview.redd.it/09lgjtq83ao21.jpg?widt...,0.0,20,mildlyinteresting,0.84,1,0,0,29923
21513,the gods have blessed this water,2018-12-02 20:45:54,i.redd.it,https://preview.redd.it/8b5rk0ukfx121.jpg?widt...,6.0,2,mildlyinteresting,0.58,1,0,0,21513
7095,lake bled,2017-01-30 14:24:58,i.imgur.com,https://external-preview.redd.it/hxLV7XRY8vITz...,9.0,626,pic,0.94,1,0,0,07095
21549,cute little batman and superman fighting,2013-11-19 21:26:28,i.imgur.com,https://external-preview.redd.it/PIcL6Wspzdqfu...,6.0,4,photoshopbattles,0.71,1,0,0,21549


In [12]:
# Ignore rows in corrupted_indices.txt files
def filter_out_corrupted_rows(split, DF):
    # File with corrupted indices
    # THIS PATH IS FOR DHRUV LAPTOP
    if split == "train":
        corrupted_indices_file = f"/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/New Corrupted File Information/{split}_corrupted_indices.txt"

    else:
        corrupted_indices_file = f"/content/drive/MyDrive/ESE 5460 Term Project/Cleaned Data/New Corrupted File Information/{split}_5k_corrupted_indices.txt"

    # Store list of corrupted indices
    corrupted_indices = None

    # Get list of corrupted indices
    with open(corrupted_indices_file, "r") as f:
        corrupted_indices = list(int(line.strip()) for line in f if line.strip())

    print(f"Split: {split}, Corrupted Indices: {corrupted_indices}, Length: {len(corrupted_indices)}")

    # Filter out corrupted rows
    DF = DF.drop(index = corrupted_indices)

    return DF

In [13]:
TRAIN_DATA = filter_out_corrupted_rows("train", TRAIN_DATA)
VALIDATION_DATA = filter_out_corrupted_rows("validation", VALIDATION_DATA)
TEST_DATA = filter_out_corrupted_rows("test", TEST_DATA)

Split: train, Corrupted Indices: [2862, 26040, 28337, 18547, 13374, 11288, 31984, 18451, 19000, 22479, 8048, 32075, 22918, 5586, 19345, 12770, 32189, 14628, 9081, 6611, 2927], Length: 21
Split: validation, Corrupted Indices: [6568, 32176], Length: 2
Split: test, Corrupted Indices: [29133, 9437, 26504, 11394], Length: 4


In [14]:
TRAIN_DATA

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label,image_num
0,this spongebob squarepants branded battery,2019-07-30 20:00:50,i.redd.it,https://preview.redd.it/f39wxxk8yhd31.jpg?widt...,4.0,33,mildlyinteresting,0.95,1,0,0,00000
1,award for careless talk,2011-09-03 17:26:23,i.imgur.com,https://external-preview.redd.it/KgPHCi1u3fY5j...,1.0,14,propagandaposters,1.00,0,1,5,00001
2,four aligned airplanes,2017-11-20 06:05:45,i.redd.it,https://preview.redd.it/88v9axk19phx.jpg?width...,24.0,198,confusing_perspective,0.98,0,2,2,00002
3,columbus discovers the new world,2019-08-28 15:40:17,i.redd.it,https://preview.redd.it/x4wzpd0am7j31.jpg?widt...,5.0,318,fakehistoryporn,0.98,0,2,2,00003
4,feed me drummmmssssssss,2014-05-09 13:23:59,i.imgur.com,https://external-preview.redd.it/yNN57loQnVhLk...,0.0,3,pareidolia,0.62,0,2,2,00004
...,...,...,...,...,...,...,...,...,...,...,...,...
33319,toothbrush holder loves its job,2018-08-27 03:49:30,i.redd.it,https://preview.redd.it/c7n0e4ot5ki11.jpg?widt...,1.0,11,pareidolia,0.92,0,2,2,33319
33320,guess how many zika cases showed up at the oly...,2016-08-26 18:51:11,npr.org,https://external-preview.redd.it/wwdKH-S3jgOG_...,2.0,46,savedyouaclick,0.91,0,2,5,33320
33321,a very fluffy cat,2016-04-16 03:04:55,i.imgur.com,https://external-preview.redd.it/iWOHMEGAS7goI...,11.0,208,photoshopbattles,0.95,1,0,0,33321
33322,found this scared little guy in the milwaukee ...,2017-11-08 19:52:08,i.imgur.com,https://external-preview.redd.it/4fr6-DmioX6Xh...,0.0,6,pareidolia,0.80,0,2,2,33322


In [15]:
VALIDATION_DATA

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label,image_num
8637,not as heartwarming as it could have been anth...,2019-09-19 17:48:33,lifestyle.clickhole.com,https://external-preview.redd.it/850kBbKdgMKfz...,0.0,15,theonion,0.86,0,2,1,08637
20669,other discussions,2013-12-09 23:58:43,NaN,http://i.dailymail.co.uk/i/pix/2013/12/09/arti...,NaN,0,psbattle_artwork,NaN,0,2,4,20669
13179,on reflection oc,2015-01-12 17:14:55,i.imgur.com,https://external-preview.redd.it/tiFw8Ggb178E4...,0.0,3,pareidolia,0.67,0,2,2,13179
20565,viet congo setting booby trap,2019-02-27 09:57:07,i.redd.it,https://preview.redd.it/mj81gkh533j21.jpg?widt...,4.0,14,fakehistoryporn,0.80,0,2,2,20565
15504,chief has some happy shoulder armour,2013-02-20 01:00:02,i.imgur.com,https://external-preview.redd.it/kqqIlnAOZxzxA...,0.0,6,pareidolia,1.00,0,2,2,15504
...,...,...,...,...,...,...,...,...,...,...,...,...
29923,albino squirrel spotted outside the national g...,2019-03-25 14:50:50,i.redd.it,https://preview.redd.it/09lgjtq83ao21.jpg?widt...,0.0,20,mildlyinteresting,0.84,1,0,0,29923
21513,the gods have blessed this water,2018-12-02 20:45:54,i.redd.it,https://preview.redd.it/8b5rk0ukfx121.jpg?widt...,6.0,2,mildlyinteresting,0.58,1,0,0,21513
7095,lake bled,2017-01-30 14:24:58,i.imgur.com,https://external-preview.redd.it/hxLV7XRY8vITz...,9.0,626,pic,0.94,1,0,0,07095
21549,cute little batman and superman fighting,2013-11-19 21:26:28,i.imgur.com,https://external-preview.redd.it/PIcL6Wspzdqfu...,6.0,4,photoshopbattles,0.71,1,0,0,21549


In [16]:
class RedditDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Fetch text
        text = row['clean_title']

        # Get image number
        image_num = row['image_num']

        # Fetch Image
        img_path = os.path.join(self.image_dir, f"{image_num}.jpg")
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Label
        label = torch.tensor(row['2_way_label'], dtype=torch.long)

        return text, image, label

In [17]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [18]:
def collate_fn(batch):
    texts, images, labels = zip(*batch)
    images = torch.stack(images)
    labels = torch.stack(labels)
    return list(texts), images, labels

In [19]:
train_dataset = RedditDataset(
    df = TRAIN_DATA,
    image_dir = "train_images",
    transform = train_tfms
)

validation_dataset = RedditDataset(
    df = VALIDATION_DATA,
    image_dir = "validation_images",
    transform = val_test_tfms
)

test_dataset = RedditDataset(
    df = TEST_DATA,
    image_dir = "test_images",
    transform = val_test_tfms
)

In [20]:
B = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=B,
    shuffle=True,
    collate_fn=collate_fn
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=B,
    shuffle=False,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=B,
    shuffle=False,
    collate_fn=collate_fn
)

In [21]:
## Sanity Check
texts, imgs, labels = next(iter(train_loader))
print("Number of texts:", len(texts))
print("Example text:", texts[0])
print("Images shape:", imgs.shape)      # (B, 3, 224, 224)
print("Labels shape:", labels.shape)
print("Labels:", labels)

Number of texts: 32
Example text: found this enticing sign on a road in texas
Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Labels: tensor([1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1])


In [22]:
# Use CPU/MPS if possible
import sys
device = None
if "google.colab" in sys.modules:
    # Running in Colab
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
else:
    # Not in Colab (e.g., Mac)
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

print("Using device:", device)

Using device: cuda


# Freezing weights for BERT and ResNet

In [23]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torchvision import models

class MultimodalClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # ----- TEXT ENCODER -----
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        bert_hidden = 768  # CLS embedding size

        # ----- IMAGE ENCODER -----
        resnet = models.resnet101(weights='IMAGENET1K_V1')
        # Remove the final FC head → get (B, 2048)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        img_hidden = 2048

        # Freeze BERT
        for param in self.bert.parameters():
            param.requires_grad = False

        # Freeze ResNet
        for param in self.cnn.parameters():
            param.requires_grad = False

        # ----- FUSION + CLASSIFICATION HEAD -----
        fusion_dim = bert_hidden + img_hidden
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, num_classes)
        )

    def forward(self, texts, images):
        # Encode text
        encoding = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt"
        ).to(images.device)

        bert_outputs = self.bert(**encoding)
        cls_emb = bert_outputs.last_hidden_state[:, 0, :]   # (B, 768)

        # Encode image
        img_feat = self.cnn(images)       # (B, 2048, 1, 1)
        img_feat = img_feat.view(img_feat.size(0), -1)  # (B, 2048)

        # Fuse
        fused = torch.cat([cls_emb, img_feat], dim=1)

        # Classify
        logits = self.classifier(fused)
        return logits

In [24]:
p0 = (TRAIN_DATA['2_way_label'] == 0).mean()
p1 = (TRAIN_DATA['2_way_label'] == 1).mean()
print(f"{p0*100:.2f}% label=0, {p1*100:.2f}% label=1")

44.63% label=0, 55.37% label=1


In [25]:
class_weights = torch.tensor([p1, p0]).float().to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
print("Class Weights:", class_weights)

Class Weights: tensor([0.5537, 0.4463], device='cuda:0')


In [26]:
model = MultimodalClassifier(num_classes=2).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:01<00:00, 169MB/s]


In [27]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [28]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, device, criterion, optimizer, num_epochs=3):
    model = model.to(device)

    # --- simple bookkeeping lists ---
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    global_step = 0

    for epoch in range(num_epochs):
        print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")
        model.train()

        for texts, images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # ---- Forward + Backward ----
            optimizer.zero_grad()
            outputs = model(texts, images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # ---- Track training metrics at every update ----
            preds = torch.argmax(outputs, dim=1)
            train_acc = (preds == labels).float().mean().item()
            train_losses.append(loss.item())
            train_accuracies.append(train_acc)

            # ---- Validate every 100 weight updates ----
            if global_step % 100 == 0:
                model.eval()
                val_loss_total = 0.0
                val_correct_total = 0
                val_samples_total = 0

                with torch.no_grad():
                    for v_texts, v_images, v_labels in val_loader:
                        v_images = v_images.to(device)
                        v_labels = v_labels.to(device)
                        v_outputs = model(v_texts, v_images)
                        v_loss = criterion(v_outputs, v_labels)

                        v_preds = torch.argmax(v_outputs, dim=1)
                        val_correct_total += (v_preds == v_labels).sum().item()
                        val_samples_total += v_labels.size(0)
                        val_loss_total += v_loss.item() * v_labels.size(0)  # sum up batch loss

                avg_val_loss = val_loss_total / val_samples_total
                avg_val_acc = val_correct_total / val_samples_total

                val_losses.append(avg_val_loss)
                val_accuracies.append(avg_val_acc)

                # save model in Milestone 3 directory
                save_dir = "Milestone #3"
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)

                torch.save(model.state_dict(), f"{save_dir}/pretrained_latest_model.pt")

                print(
                    f"[Update {global_step}] "
                    f"Train Loss: {loss.item():.4f}, Train Acc: {train_acc:.4f} | "
                    f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}"
                )
                model.train()

            global_step += 1

    # save model in Milestone 3 directory
    save_dir = "Milestone #3"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    torch.save(model.state_dict(), f"{save_dir}/pretrained_latest_model.pt")

    return train_losses, train_accuracies, val_losses, val_accuracies

In [29]:
train_losses, train_accs, val_losses, val_accs = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=validation_loader,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=3
)


===== Epoch 1/3 =====


  0%|          | 0/1041 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
  0%|          | 1/1041 [00:55<15:54:42, 55.08s/it]

[Update 0] Train Loss: 0.6919, Train Acc: 0.5625 | Val Loss: 0.7193, Val Acc: 0.4694


 10%|▉         | 101/1041 [02:25<4:20:48, 16.65s/it]

[Update 100] Train Loss: 0.6683, Train Acc: 0.5625 | Val Loss: 0.6712, Val Acc: 0.5812


...

 88%|████████▊ | 919/1041 [14:36<33:09, 16.30s/it]

[Update 3000] Train Loss: 0.4943, Train Acc: 0.7188 | Val Loss: 0.5062, Val Acc: 0.7559


 98%|█████████▊| 1019/1041 [16:09<06:30, 17.75s/it]

[Update 3100] Train Loss: 0.5638, Train Acc: 0.6562 | Val Loss: 0.5042, Val Acc: 0.7595


100%|██████████| 1041/1041 [16:17<00:00,  1.06it/s]


In [30]:
import pickle

with open("multimodal_pretrained_metrics.pkl", "wb") as f:
    pickle.dump({
        "train_losses": train_losses,
        "train_accs": train_accs,
        "val_losses": val_losses,
        "val_accs": val_accs
    }, f)

In [31]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(logits, labels):
    logits = np.array(logits)
    labels = np.array(labels)
    predictions = np.argmax(logits, axis=-1)

    return {
        "accuracy": accuracy_score(labels, predictions),

        "pos_precision": precision_score(labels, predictions, pos_label=1, average="binary", zero_division=0),
        "pos_recall": recall_score(labels, predictions, pos_label=1, average="binary", zero_division=0),
        "pos_f1": f1_score(labels, predictions, pos_label=1, average="binary", zero_division=0),

        "neg_precision": precision_score(labels, predictions, pos_label=0, average="binary", zero_division=0),
        "neg_recall": recall_score(labels, predictions, pos_label=0, average="binary", zero_division=0),
        "neg_f1": f1_score(labels, predictions, pos_label=0, average="binary", zero_division=0),

        "f1_macro": f1_score(labels, predictions, average="macro"),
        "f1_micro": f1_score(labels, predictions, average="micro"),
        "f1_weighted": f1_score(labels, predictions, average="weighted"),
    }

In [32]:
def evaluate_model(model, dataloader):
    model.eval()
    all_logits = []
    all_labels = []

    with torch.no_grad():
        for txts, imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(txts, imgs)      # logits
            all_logits.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return compute_metrics(all_logits, all_labels)

In [33]:
# ---- evaluate ----
train_metrics = evaluate_model(model, train_loader)
val_metrics   = evaluate_model(model, validation_loader)
test_metrics  = evaluate_model(model, test_loader)

# ---- save to CSV ----
pd.DataFrame([train_metrics]).to_csv("Multimodal_pretrained_train_metrics.csv", index=False)
pd.DataFrame([val_metrics]).to_csv("Multimodal_pretrained_val_metrics.csv", index=False)
pd.DataFrame([test_metrics]).to_csv("Multimodal_pretrained_test_metrics.csv", index=False)

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


# Finetuned model

In [34]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torchvision import models

class FineMultimodalClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # ----- TEXT ENCODER -----
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        bert_hidden = 768  # CLS embedding size

        # ----- IMAGE ENCODER -----
        resnet = models.resnet101(weights='IMAGENET1K_V1')
        resnet.fc = nn.Identity()
        # Remove the final FC head → get (B, 2048)
        self.cnn = resnet
        img_hidden = 2048

        # ----- FUSION + CLASSIFICATION HEAD -----
        fusion_dim = bert_hidden + img_hidden
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, num_classes)
        )

    def forward(self, texts, images):
        # Encode text
        encoding = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt"
        ).to(images.device)

        bert_outputs = self.bert(**encoding)
        cls_emb = bert_outputs.last_hidden_state[:, 0, :]   # (B, 768)

        # Encode image
        img_feat = self.cnn(images)       # (B, 2048, 1, 1)
        img_feat = img_feat.view(img_feat.size(0), -1)  # (B, 2048)

        # Fuse
        fused = torch.cat([cls_emb, img_feat], dim=1)

        # Classify
        logits = self.classifier(fused)
        return logits

In [35]:
p0 = (TRAIN_DATA['2_way_label'] == 0).mean()
p1 = (TRAIN_DATA['2_way_label'] == 1).mean()
print(f"{p0*100:.2f}% label=0, {p1*100:.2f}% label=1")

44.63% label=0, 55.37% label=1


In [36]:
class_weights = torch.tensor([p1, p0]).float().to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
print("Class Weights:", class_weights)

Class Weights: tensor([0.5537, 0.4463], device='cuda:0')


In [37]:
model = FineMultimodalClassifier(num_classes=2).to(device)

In [38]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [39]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, device, criterion, optimizer, num_epochs=3):
    model = model.to(device)

    # --- simple bookkeeping lists ---
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    global_step = 0

    for epoch in range(num_epochs):
        print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")
        model.train()

        for texts, images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # ---- Forward + Backward ----
            optimizer.zero_grad()
            outputs = model(texts, images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # ---- Track training metrics at every update ----
            preds = torch.argmax(outputs, dim=1)
            train_acc = (preds == labels).float().mean().item()
            train_losses.append(loss.item())
            train_accuracies.append(train_acc)

            # ---- Validate every 100 weight updates ----
            if global_step % 100 == 0:
                model.eval()
                val_loss_total = 0.0
                val_correct_total = 0
                val_samples_total = 0

                with torch.no_grad():
                    for v_texts, v_images, v_labels in val_loader:
                        v_images = v_images.to(device)
                        v_labels = v_labels.to(device)
                        v_outputs = model(v_texts, v_images)
                        v_loss = criterion(v_outputs, v_labels)

                        v_preds = torch.argmax(v_outputs, dim=1)
                        val_correct_total += (v_preds == v_labels).sum().item()
                        val_samples_total += v_labels.size(0)
                        val_loss_total += v_loss.item() * v_labels.size(0)  # sum up batch loss

                avg_val_loss = val_loss_total / val_samples_total
                avg_val_acc = val_correct_total / val_samples_total

                val_losses.append(avg_val_loss)
                val_accuracies.append(avg_val_acc)

                # save model in Milestone 3 directory
                save_dir = "Milestone #3"
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)

                torch.save(model.state_dict(), f"{save_dir}/finetuned_latest_model.pt")

                print(
                    f"[Update {global_step}] "
                    f"Train Loss: {loss.item():.4f}, Train Acc: {train_acc:.4f} | "
                    f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}"
                )
                model.train()

            global_step += 1

    # save model in Milestone 3 directory
    save_dir = "Milestone #3"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    torch.save(model.state_dict(), f"{save_dir}/finetuned_latest_model.pt")
    return train_losses, train_accuracies, val_losses, val_accuracies

In [40]:
train_losses, train_accs, val_losses, val_accs = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=validation_loader,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=3
)


===== Epoch 1/3 =====


  0%|          | 0/1041 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
  0%|          | 1/1041 [00:57<16:30:55, 57.17s/it]

[Update 0] Train Loss: 0.7287, Train Acc: 0.5938 | Val Loss: 0.7005, Val Acc: 0.5230


 10%|▉         | 101/1041 [03:19<4:59:51, 19.14s/it]

[Update 100] Train Loss: 0.3924, Train Acc: 0.7500 | Val Loss: 0.4566, Val Acc: 0.7993


...

 88%|████████▊ | 919/1041 [22:44<40:19, 19.83s/it]

[Update 3000] Train Loss: 0.1548, Train Acc: 0.9375 | Val Loss: 0.4216, Val Acc: 0.8457


 98%|█████████▊| 1019/1041 [25:11<07:13, 19.71s/it]

[Update 3100] Train Loss: 0.1887, Train Acc: 0.9062 | Val Loss: 0.4367, Val Acc: 0.8501


100%|██████████| 1041/1041 [25:29<00:00,  1.47s/it]


In [41]:
import pickle

with open("multimodal_finetuned_metrics.pkl", "wb") as f:
    pickle.dump({
        "train_losses": train_losses,
        "train_accs": train_accs,
        "val_losses": val_losses,
        "val_accs": val_accs
    }, f)

In [42]:
# ---- evaluate ----
train_metrics = evaluate_model(model, train_loader)
val_metrics   = evaluate_model(model, validation_loader)
test_metrics  = evaluate_model(model, test_loader)

# ---- save to CSV ----
pd.DataFrame([train_metrics]).to_csv("Multimodal_finetuned_train_metrics.csv", index=False)
pd.DataFrame([val_metrics]).to_csv("Multimodal_finetuned_val_metrics.csv", index=False)
pd.DataFrame([test_metrics]).to_csv("Multimodal_finetuned_test_metrics.csv", index=False)

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
